In [1]:
import pandas as pd
from functions import util
import os
import warnings

warnings.filterwarnings("ignore")

## 1.Backfill feature pipeline

Write a backfill feature pipeline that downloads historical weather data (ideally >1
year of data), loads a csv file with historical air quality data (downloaded from
https://aqicn.org) and registers them as 2 Feature Groups with Hopsworks.

### 1.1 Import air quality data on 5 years at Stockholm

In [2]:
air_quality_df = pd.read_csv("../../data/stockholm-hornsgatan 108 gata-air-quality.csv")

air_quality_df['date'] = pd.to_datetime(air_quality_df['date'], format='%Y/%m/%d')

# Handle non-numeric values in 'pm25' and 'pm10', replace them with NaN
air_quality_df['pm25'] = pd.to_numeric(air_quality_df['pm25'], errors='coerce')
air_quality_df['pm10'] = pd.to_numeric(air_quality_df['pm10'], errors='coerce')

# Fill NaN values with 0 (or use mean or another method depending on your choice)
air_quality_df['pm25'].fillna(0, inplace=True)
air_quality_df['pm10'].fillna(0, inplace=True)

# Convert 'pm25' and 'pm10' columns to integers
air_quality_df['pm25'] = air_quality_df['pm25'].astype(int)
air_quality_df['pm10'] = air_quality_df['pm10'].astype(int)


air_quality_df = air_quality_df.drop(columns=['no2'])


air_quality_df.head()

,date,pm25,pm10
0,2024-11-01,8,3
1,2024-11-02,5,3
2,2024-11-03,6,6
3,2024-11-04,7,4
4,2024-11-05,11,6


### 1.2 Import weather data on one year at Stockholm

In [3]:
import pandas as pd
from datetime import datetime, timedelta

# Coordinates of Stockholm and date range for historical data
latitude = 59.3293
longitude = 18.0686
today = datetime.today().date()
start_date = today - timedelta(days=365)
end_date = today - timedelta(days=2)

country="sweden"
city = "stockholm"
street = "stockholm-hornsgatan-108"
aqicn_url="https://api.waqi.info/feed/@10009"

earliest_aq_date = pd.Series.min(air_quality_df['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')

# Calculate daily average temperature
df_weather = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

# Print the result
df_weather = df_weather.drop(columns='city')

df_weather



Coordinates 59.29701232910156°N 18.163265228271484°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
0,2017-10-03,14.615000,10.214999,0.500000,18.391737,209.271942
1,2017-10-04,13.565000,7.415000,2.900000,22.206486,248.656326
2,2017-10-05,11.014999,5.965000,2.600000,16.595179,306.521240
3,2017-10-06,11.165000,6.215000,0.500000,23.871555,320.408325
4,2017-10-07,8.815000,4.215000,6.300001,14.332341,343.018829
...,...,...,...,...,...,...
2595,2024-11-10,7.065000,4.415000,0.100000,9.292255,145.380463
2596,2024-11-11,5.665000,3.615000,1.100000,9.676569,234.773972
2597,2024-11-12,4.965000,1.315000,0.000000,10.990322,288.903290
2598,2024-11-13,5.765000,0.065000,0.000000,15.124284,238.321777


### 1.3 Connection to hopsworks

In [4]:
import hopsworks

with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

In [5]:
project = hopsworks.login(project="ID2223LAB1KTH")
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1170583
2024-11-16 11:58:13,998 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Create a weather data feature group
weather_fg = fs.get_or_create_feature_group(
    name="stockholm_weather",
    version=1,
    description="Weather data for Stockholm including temperature, humidity, wind speed, and wind direction",
    primary_key=["date"],  
    event_time="date" 
)


#weather_fg.insert(df_weather)

print("Weather feature group created and data inserted successfully.")


Weather feature group created and data inserted successfully.


In [7]:

air_quality_fg = fs.get_or_create_feature_group(
    name="stockholm_air_quality",
    version=1,
    description="Air quality data for Stockholm with PM2.5 concentrations",
    primary_key=["date"],  # 'time' column as the primary key
    event_time="date"      # Specify 'time' as the event time
)


#air_quality_fg.insert(air_quality_df)

print("Air quality feature group created and data inserted successfully.")


Air quality feature group created and data inserted successfully.
